# ResNet Variants (Not Done Yet)
Merge ResNet and Linear regression in one architecture to combine NHTS and Images. The training result is very bad.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
import torch.optim as optim

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import util
import statsmodels.api as sm
from scipy import stats
import copy

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# ALWAYS choose devise first.
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Helper Fuctions

In [3]:
# This is exactly the same as 2_cnn initialize_data
def initialize_data(image_type, output_var, output_type, input_var, BE_var, num_categories, size):
    # inputs: image_list, output_var, output_type.
    # outputs: data loader for training and testing.
    
    ### read image array
    if image_type == 'rgb':
        image_array_ = np.load("../data_process/image_array_rgb_tract_large.npy", mmap_mode='r')
        image_array = image_array_[:size,]
    elif image_type == 'bw':
        image_array_ = np.load("../data_process/image_array_bw_tract_large.npy", mmap_mode='r')
        image_array = image_array_[:size,]        
    elif image_type == 'merge':
        bw_image_array_ = np.load("../data_process/image_array_bw_tract_large.npy", mmap_mode='r')
        rgb_image_array_ = np.load("../data_process/image_array_rgb_tract_large.npy", mmap_mode='r')
        bw_image_array = bw_image_array_[:size,]
        rgb_image_array = rgb_image_array_[:size,]
        image_array = np.concatenate([rgb_image_array, bw_image_array], axis=1)
    
    ### create output array
    df_ = pd.read_csv("../data_process/df_merged_tract_large.csv")
    df = df_.iloc[:size,]
    y_ = df[output_var].values 
    # cut y into categories for discrete variables
    if output_type == 'continuous':
        y = copy.deepcopy(y_)
    elif output_type == 'discrete':
        y = np.array(pd.qcut(y_, q = num_categories, labels=np.arange(num_categories))) 
    x = df[input_var]
    BE = df[BE_var]
            
    ### randomization
    shuffle_idx = np.arange(size)
    np.random.seed(0)
    np.random.shuffle(shuffle_idx)
    train_ratio = 0.8

    ###
    # y
    if output_type == 'discrete':
        y_train = y[shuffle_idx[:int(train_ratio*size)]].astype("int")
        y_test = y[shuffle_idx[int(train_ratio*size):]].astype("int")
    elif output_type == 'continuous':
        y_train = y[shuffle_idx[:int(train_ratio*size)]].astype("float32")
        y_test = y[shuffle_idx[int(train_ratio*size):]].astype("float32")
    # BE
    BE_train = BE.values[shuffle_idx[:int(train_ratio*size)]].astype("float32")
    BE_test = BE.values[shuffle_idx[int(train_ratio*size):]].astype("float32")        
    # image array
    x_train_images = image_array[shuffle_idx[:int(train_ratio*size)],].astype("float32")
    x_test_images = image_array[shuffle_idx[int(train_ratio*size):],].astype("float32")
    # NHTS
    x_train = x.values[shuffle_idx[:int(train_ratio*size)]].astype("float32")
    x_test = x.values[shuffle_idx[int(train_ratio*size):]].astype("float32")
    
    return y_train,y_test,BE_train,BE_test,x_train,x_test,x_train_images,x_test_images


num_categories=1
num_bottleneck=1
model = bottleneck_resnet18(num_categories, num_bottleneck)

model.parameters()

In [4]:
# model: combine resnet and linear
class ResNet_And_Linear(nn.Module):
    def __init__(self, num_categories, num_x_nhts, input_channels = 3, use_pretrained=True, full_training=True):
        super(ResNet_And_Linear, self).__init__()
        # resnet branch
        self.resnet_branch = models.resnet18(pretrained=use_pretrained)
        for param in self.resnet_branch.parameters():
            param.requires_grad=full_training
        if input_channels != 3:
            self.resnet_branch.conv1 = nn.Conv2d(input_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        num_ftrs = self.resnet_branch.fc.in_features
        self.resnet_branch.fc = nn.Linear(num_ftrs, num_categories)
        # linear branch
        self.linear_branch = nn.Linear(num_x_nhts, num_categories)
    
    def forward(self, x_images, x_nhts):
        y_from_images = self.resnet_branch(x_images)
        y_from_nhts = self.linear_branch(x_nhts)
        y = y_from_images + y_from_nhts
        return y


In [5]:
# This is quite similar to 2_cnn initialize_data
def train_continuous_merged_model(model, train_dl, test_dl, criterion, optimizer, device, total_mse_train, total_mse_test, n_epoch = 25):
    mse_train_list = []
    mse_test_list = []
    r_square_train_list = []
    r_square_test_list = []

    # automatic model searching.
    best_model_wts = copy.deepcopy(model.state_dict())
    best_r_square = 0.0
    
    for epoch in range(n_epoch): # loop over the dataset multiple times
        running_mse_train = 0.0
        running_mse_test = 0.0
        
        # training
        for x_train_images_batch, x_train_nhts_batch, y_train_batch in train_dl:
            # to device
            x_train_images_batch = x_train_images_batch.to(device)
            x_train_nhts_batch = x_train_nhts_batch.to(device)
            y_train_batch = y_train_batch.to(device)

            # forward + backward
            outputs = model(x_train_images_batch, x_train_nhts_batch)
            # sw: be careful about the dimension matching at this point...
            loss = criterion(outputs.view(-1), y_train_batch) # this .view(-1) seems specific to continuous variables
            loss.backward()

            # statistics
            running_mse_train += loss.item()*batch_size

            # optimize
            with torch.no_grad():
                optimizer.step()
                optimizer.zero_grad()

        # testing
        for x_test_images_batch, x_test_nhts_batch, y_test_batch in test_dl:
            # to device
            x_test_images_batch = x_test_images_batch.to(device)
            x_test_nhts_batch = x_test_nhts_batch.to(device)
            y_test_batch = y_test_batch.to(device)

            # forward + backward
            outputs = model(x_test_images_batch, x_test_nhts_batch)
            loss = criterion(outputs.view(-1), y_test_batch) # this .view(-1) seems specific to continuous variables
            running_mse_test += loss.item()*batch_size # this *batch_size is specific to continuous variables.

        running_r_square_train = 1-running_mse_train/total_mse_train.item()
        running_r_square_test = 1-running_mse_test/total_mse_test.item()
        
        print("Epoch {}: Training MSE {}; Testing MSE {}".format(epoch, running_mse_train, running_mse_test))
        print("Epoch {}: Training R2 {}; Testing R2 {}".format(epoch, running_r_square_train, running_r_square_test))

        mse_train_list.append(running_mse_train)
        mse_test_list.append(running_mse_test)
        r_square_train_list.append(running_r_square_train)
        r_square_test_list.append(running_r_square_test)

        # check overfitting for early stopping.
        #if epoch > 5 and running_r_square_test < 0.0:
        #    break # break the for loop.
        
        # replace the model with the best performance.
        if running_r_square_test > best_r_square:
            best_r_square = running_r_square_test
            best_model_wts = copy.deepcopy(model.state_dict())

    # load the best model weights
    model.load_state_dict(best_model_wts)
    return model, mse_train_list, mse_test_list, r_square_train_list, r_square_test_list


In [6]:
### 
output_list = ['HHVEHCNT_mean_norm', 'HHVEHCNT_P_CAP_mean_norm', 'TRPTRANS_1_mean_norm', 'TRPTRANS_2_mean_norm', 'TRPTRANS_3_mean_norm']
input_var=['R_AGE_IMP_mean', 'HHSIZE_mean', 'HHFAMINC_mean', 'HBHTNRNT_mean', 'HBPPOPDN_mean', 'HBRESDN_mean', 
           'R_SEX_IMP_2_mean', 'EDUC_2_mean', 'HH_RACE_2_mean', 'HOMEOWN_1_mean', 'HOMEOWN_2_mean',
           'HBHUR_R_mean', 'HBHUR_S_mean', 'HBHUR_T_mean','HBHUR_U_mean']
BE_var = ['density', 'diversity', 'design']
image_type = 'bw' # 'rgb', 'bw'
output_type = 'continuous'
num_categories = 1 # (1) certain category values can cause errors. (2) when output_type = 'continuous', this value needs to be 1.
size = 12000 # size needs to be smaller than the max (18491)

# output_var = 'TRPTRANS_1_mean_norm'

performance = {}
model_dic = {}

In [7]:
for output_var in output_list:
    print(output_var)
    #     
    y_train,y_test,BE_train,BE_test,x_train,x_test,x_train_images,x_test_images = \
        initialize_data(image_type, output_var, output_type, input_var, BE_var, num_categories, size)

    # data set challenge for the concatenated datasets.
    x_train_images_norm = x_train_images/255
    x_test_images_norm = x_test_images/255

    x_train_images_torch = torch.from_numpy(x_train_images_norm)
    x_test_images_torch = torch.from_numpy(x_test_images_norm)
    y_train_torch = torch.from_numpy(y_train)
    y_test_torch = torch.from_numpy(y_test)

    x_train_nhts_torch = torch.from_numpy(x_train)
    x_test_nhts_torch = torch.from_numpy(x_test)

    batch_size = 100
    train_ds = TensorDataset(x_train_images_torch, x_train_nhts_torch, y_train_torch)
    train_dl = DataLoader(train_ds, batch_size, shuffle = True)

    batch_size = 100
    test_ds = TensorDataset(x_test_images_torch, x_test_nhts_torch, y_test_torch)
    test_dl = DataLoader(test_ds, batch_size, shuffle = True)    
    
    # model set up 
    num_x_nhts = x_train.shape[1]
    input_channels = 4
    use_pretrained = True
    full_training = True
    model_name = 'resnet_and_linear'
    model = ResNet_And_Linear(num_categories, num_x_nhts, input_channels, use_pretrained, full_training)
    model.to(device)

    # training set up.
    criterion = nn.MSELoss(reduction='mean')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    n_epoch = 25

    # create baseline mse    
    total_mse_train = criterion(y_train_torch.mean().repeat(y_train_torch.size()), y_train_torch)*y_train_torch.size()[0]
    total_mse_test = criterion(y_test_torch.mean().repeat(y_test_torch.size()), y_test_torch)*y_test_torch.size()[0]
    print(total_mse_train)
    print(total_mse_test)
    
    # training here
    model, mse_train_list, mse_test_list, r_square_train_list, r_square_test_list = \
        train_continuous_merged_model(model, train_dl, test_dl, criterion, optimizer, device, total_mse_train, total_mse_test, n_epoch)

    # save models.
    PATH = './models/'+model_name+'_'+output_var+'_'+image_type+'.pth'
    torch.save(model.state_dict(), PATH)
    model_dic[output_var]=model.state_dict()
    
    # save performance
    performance[output_var] = {}
    performance[output_var]['mse_train_list']=mse_train_list
    performance[output_var]['mse_test_list']=mse_test_list
    performance[output_var]['r_square_train_list']=r_square_train_list
    performance[output_var]['r_square_test_list']=r_square_test_list
    
    # for i, (x_images_batch, x_nhts_batch, y_batch) in enumerate(train_concat_dl):
    #     print(i)


HHVEHCNT_mean_norm
tensor(10700.4521)
tensor(2549.4304)
Epoch 0: Training MSE 4961943844.433594; Testing MSE 220687191.015625
Epoch 0: Training R2 -463712.4744963227; Testing R2 -86562.33167248697
Epoch 1: Training MSE 470774952.734375; Testing MSE 88693916.796875
Epoch 1: Training R2 -43994.80000954618; Testing R2 -34788.698947576275
Epoch 2: Training MSE 297294765.9667969; Testing MSE 66993853.515625
Epoch 2: Training R2 -27782.3835283501; Testing R2 -26276.96898951177
Epoch 3: Training MSE 230535849.75585938; Testing MSE 54315483.7890625
Epoch 3: Training R2 -21543.496116411552; Testing R2 -21303.94849540822
Epoch 4: Training MSE 170203946.6796875; Testing MSE 37459639.0625
Epoch 4: Training R2 -15905.238756886642; Testing R2 -14692.336507551328
Epoch 5: Training MSE 121546618.01757812; Testing MSE 25528741.430664062
Epoch 5: Training R2 -11358.017014559202; Testing R2 -10012.507813814495
Epoch 6: Training MSE 88242155.22460938; Testing MSE 20932307.91015625
Epoch 6: Training R2 -82

Epoch 6: Training MSE 8975443.963623047; Testing MSE 1399000.8483886719
Epoch 6: Training R2 -770.1185337885894; Testing R2 -459.34652633358166
Epoch 7: Training MSE 3368391.6427612305; Testing MSE 528622.5921630859
Epoch 7: Training R2 -288.39284065713156; Testing R2 -172.94526552575942
Epoch 8: Training MSE 1310119.18258667; Testing MSE 200173.72665405273
Epoch 8: Training R2 -111.55790657922347; Testing R2 -64.86792269252439
Epoch 9: Training MSE 555497.3907470703; Testing MSE 98823.67420196533
Epoch 9: Training R2 -46.72514153198028; Testing R2 -31.518304181726542
Epoch 10: Training MSE 284247.78594970703; Testing MSE 53065.4486656189
Epoch 10: Training R2 -23.420935256523254; Testing R2 -16.461386810227225
Epoch 11: Training MSE 178331.05239868164; Testing MSE 38469.850158691406
Epoch 11: Training R2 -14.321178563643791; Testing R2 -11.658649856806171
Epoch 12: Training MSE 134458.09497833252; Testing MSE 29792.188453674316
Epoch 12: Training R2 -10.551866345098958; Testing R2 -8.

Epoch 13: Training MSE 4078417.839050293; Testing MSE 1644220.736694336
Epoch 13: Training R2 -354.65803271593285; Testing R2 -562.2450764037035
Epoch 14: Training MSE 2852571.6705322266; Testing MSE 1335162.466430664
Epoch 14: Training R2 -247.7582362965883; Testing R2 -456.37392105151355
Epoch 15: Training MSE 2149336.072540283; Testing MSE 975821.2188720703
Epoch 15: Training R2 -186.43264407235742; Testing R2 -333.2778038944847
Epoch 16: Training MSE 1585722.7020263672; Testing MSE 750359.912109375
Epoch 16: Training R2 -137.28279467486325; Testing R2 -256.04366609316776
Epoch 17: Training MSE 1204948.892211914; Testing MSE 561194.0826416016
Epoch 17: Training R2 -104.07745146267915; Testing R2 -191.24292511373244
Epoch 18: Training MSE 904579.6272277832; Testing MSE 473391.9189453125
Epoch 18: Training R2 -77.88377879635347; Testing R2 -161.16537208459772
Epoch 19: Training MSE 752778.7567138672; Testing MSE 376519.9531555176
Epoch 19: Training R2 -64.64599858300548; Testing R2 -1

In [8]:
# save
import pickle
with open('outputs/performance_continuous_'+image_type+'_'+model_name+'.pickle', 'wb') as h:
    pickle.dump(performance, h, protocol=pickle.HIGHEST_PROTOCOL)